# NER 학습 데이터 생성기

각 크롤러에서 얻은 데이터를 기반으로 

1. 텍스트 클리닝
2. mecab 품사태깅
3. Pororo 개체명 인식

의 순서로 개체명 인식기 학습데이터를 생성

## 패키지 설치 및 Import

전처리 및 데이터 생성기
* KoNLpy: pip install
* langid: 언어탐지
* Pororo: PyTorch 버전과 Split_Sentence 생략을 위해 별도 설치


KoBERT 모델
* sentencepiece: Tokenizer 패키지
* seqeval: Evaluation 패키지
* transformers: KoBERT 패키지


In [ ]:
!pip install langid jamo pandas sentencepiece transformers seqeval

In [ ]:
!pip install git+https://github.com/tsfo1489/pororo.git

In [ ]:
import re
import json
import pickle
import langid
import pandas as pd
from jamo import j2h
from pororo import Pororo
from tqdm.notebook import tqdm

## 데이터 불러오기

크롤링 데이터를 각 문장별로 분할한 데이터
원본데이터는 Json List로 구성되어 있으며 반드시 'body'를 가지고 있어야함

`
[
    {
        ...
        'body': '예시 문장 1'
        ...
    },
    {
        ...
        'body': '예시 문장 2'
        ...
    }
]
`

### 뉴스 데이터

In [ ]:
crawling_data_df = pd.read_json('./RawData/news_keyword.json', dtype={'data_id': str, 'body': str, 'keyword': str})
crawling_data_df

### 트위터 데이터

In [ ]:
crawling_data_df = pd.read_json('./RawData/twitter_cleaning.json').dropna()
crawling_data_df

### 유튜브 데이터

In [ ]:
crawling_data_df = pd.read_json('./RawData/Youtube_Ko_Raw.json', dtype={'data_id': str, 'body': str, 'keyword': str})
crawling_data_df

## 텍스트 클리닝

#### 텍스트 클리너

In [ ]:
class TextCleaner():
    '''
    Text Cleaner
    
    Remove Noise by regular expression and extract special tokens
    Check language and Correct spelling and spacing
    
    Attributes:
        cfg (List[Dict[str: str]]): Cleaning Rule List
        spacing_model (soyspacing.countbase.CountSpace): Spacing correction model
    
    methods
     * load_spacing_model: load soyspacing model
     * get_lang: get language code of sentence
     * remove_stopwords: 
     * cleaning_data: 
    '''
    def __init__(
                 self, 
                 path="./cleaning/CleaningRule.cfg", 
                 verbose=False):
        '''
        Text Cleaner initilizer
        
        Args:
            path (str, default: './cleaning/CleaningRule.cfg'): 
                Cleaning rule cfg file path
                Cleaning Rule File: JSON format
                List[Dict('name': str, 'type': str, 'regex': str, 'target': str)]
                * name (str): rule name
                * type (str): 
                    * "sub": replace "regex" to "target"
                    * "extract": extract "regex" to ret_dict[target]
                * regex (str): Regular expression of step
                * target (str): Target of cleaning Rule
                
                ex) Cleaning 'Very Funny, 😀😀😀'
                    {
                        'name': 'Remove Emoji',
                        'type': 'extract',
                        'regex': '[\\u2700-\\u27BF]|[\\u2011-\\u26FF]|[\\u2900-\\u2b59]'
                        'target': 'emoji'
                    }
                    
                    Method cleaning_data returns
                    ('True', 'Very Funny,', {'emoji': ['😀']}
            verbose (boolean, default: True): Enable debug mode
                print debug log to stdout
        '''
        with open(path, encoding='utf-8') as jsonfile:
            self.cfg = json.load(jsonfile)
        if verbose:
            for rule in self.cfg:
                print(rule)
        self.spacing_model = None
        
    def get_lang(self, sentence):
        '''
        Get language id from sentence
        
        Args:
            sentence (str): Input sentence
        
        Returns:
            language_id (str): ISO Language id 
                ex)'ko', 'en'
        '''
        lang = langid.classify(sentence)
        return lang[0]
    
    def cleaning_data(self, sent: str, verbose=False):
        '''
        Cleaning Sentence
        
        Args:
            sent (str): target sentence
            verbose (bool, default: False): Enable debug log print to stdout
        
        Returns: 
            It has two different return type
                True , sentence , ret_dict : 
                    if cleaning success and not filtered by sum options,
                    sentence (str): clean sentence 
                    ret_dict (Dict[str: List[str]]): Extracted strings
                False, reason: 
                    reason (str): Filtered reason
        '''
        

        # Merge splitted ja, mo
        # ex) ㅇㅖㅅㅣ -> 예시
        splited_word = re.findall(r'[ㄱ-ㅎ][ㅏ-ㅣ]', sent)
        for k in range(len(splited_word)):
            if verbose:
                print(f'Merge Jamo: {splited_word[k]} -> {j2h(splited_word[k][0], splited_word[k][1])}')
            sent = re.sub(splited_word[k], j2h(splited_word[k][0], splited_word[k][1]), sent)

        ret_dict = {}
        # Cleaning Step
        for idx, step in enumerate(self.cfg):
            if verbose:
                print(f'Step #{idx + 1}: {step["name"]}')
            # mode 'sub' replace regex to target
            if step['type'] == 'sub':
                sent = re.sub(step['regex'], step['target'], sent)
            # mode 'extract' extract regex to ret_dict[target]
            if step['type'] == 'extract':
                if step['target'] not in ret_dict:
                    ret_dict[step['target']] = []
                ret_dict[step['target']] += re.findall(step['regex'], sent)
                if verbose:
                    print(f'Extract: {re.findall(step["regex"], sent)}')
                sent = re.sub(step['regex'], '', sent)
            if verbose:
                print('->{}'.format(sent))

        # Languages Detection
        # Split sentence and Detect language for each part
        # Only "ko", "ko" pair can pass
        sentence_len = len(sent)
        pivot = sentence_len // 2
        lang1 = self.get_lang(sent[:pivot])
        lang2 = self.get_lang(sent[pivot:])
        if verbose:
            print('Language Detection')
            print(f'Part 1\'s Language: {lang1}')
            print(f'Part 2\'s Language: {lang2}')
        ret_dict['lang'] = [lang1, lang2]
        if ret_dict['lang'][0] != 'ko' or ret_dict['lang'][1] != 'ko':
            return False, f'Filter by Language: {lang1} {lang2} | {sent}', {}

        return True, sent, ret_dict

#### 전처리 및 언어 필터링

In [ ]:
cleaner = TextCleaner('./cleaning/CleaningRule.cfg')
crawling_data_df['spacing'] = [None for _ in range(len(crawling_data_df))]
crawling_data_df['desc'] = [None for _ in range(len(crawling_data_df))]
correct_sent = []
error_sent = [] 
for idx, row in tqdm(crawling_data_df.iterrows(), total=len(crawling_data_df), desc='Cleaning'):
    flag, x, _ = cleaner.cleaning_data(row['body'])
    row['spacing'] = flag
    row['desc'] = x
    if flag:
        correct_sent.append(x)
    else:
        error_sent.append(x)
print(len(correct_sent), len(error_sent))

#### 필터링된 문장을 pd.DataFrame으로 변환

In [ ]:
crawling_data_df = pd.DataFrame(correct_sent, columns=['body'])
crawling_data_df

## Mecab과 Pororo를 이용해 크롤링 통합 데이터 태깅

In [ ]:
class DataMaker:
    def __init__(self, device='cuda:0'):
        self.pos_tagger = Pororo('pos', 'ko', device=device)
        self.ner_tagger = Pororo('ner', 'ko', device=device)
        self.dp_tagger = Pororo('dep_parse', 'ko', device=device)
        self.NER_Transform = {
            'PERSON': 'PER',
            'LOCATION': 'LOC',
            'CITY': 'LOC',
            'COUNTRY': 'LOC',
            'ORGANIZATION': 'ORG',
            'ARTIFACT': 'AFW',
            'DATE': 'DAT',
            'TIME': 'TIME',
            'CIVILIZATION': 'O',
            'ANIMAL': 'O',
            'PLANT': 'O',
            'QUANTITY': 'O',
            'STUDY_FIELD': 'FLD',
            'THEORY': 'O',
            'EVENT': 'EVT',
            'MATERIAL': 'O',
            'TERM': 'O',
            'OCCUPATION': 'O',
            'DISEASE': 'O',
            'O': 'O'
        }
    
    def parse(self, text):
        result_str = f'#RawSentence: {text}\n'
        row = {'body': text}
        row['pos'] = self.pos_tagger(text, return_surface=True) # 품사태깅
        row['ner'] = self.ner_tagger(text)#개체명 태깅
        row['dp'] = self.dp_tagger(text)# 디팬던시 파싱
        # print(row['pos'])
        # print(row['ner'])
        # print(row['dp'])
        ner_idx = 0
        ner_bio = True
        ner_str = row['ner'][0][0]
        e_id = 1
        eojeol_str = row['dp'][0][3]
        json_data = {'RawSentence': text, 'data': []}
        for idx in range(len(row['pos'])): # 품사기준으로 제일작은거로 쪼갬
            ner_tag = self.NER_Transform[row["ner"][ner_idx][1]]
            if row['pos'][idx][1] != 'SPACE':
                # 어절 ID, 형태소 ID
                result_str += f'{e_id}\t{idx + 2 - e_id}\t'
                morph = {}
                morph['e_id'] = e_id
                morph['m_id'] = idx + 2 - e_id
                # 형태소
                result_str += f'{row["pos"][idx][0]}\t'
                morph['morph'] = row["pos"][idx][0]
                # 품사
                result_str += f'{row["pos"][idx][1]}\t'
                morph['pos'] = row["pos"][idx][1]
                # NER
                result_str += f'{ner_tag}'
                morph['ner'] = ner_tag
                if ner_tag != 'O':
                    result_str += '-B\t' if ner_bio else '-I\t'
                    morph['ner'] += '-B' if ner_bio else '-I'
                    if ner_bio:
                        ner_bio = False
                else:
                    result_str += '\t'
                # DP
                if idx + 1 < len(row['pos']) and row['pos'][idx + 1][1] == 'SPACE':
                    result_str += f'{row["dp"][e_id - 1][2]}\t'
                    morph['dp'] = row["dp"][e_id - 1][2]
                    result_str += f'{eojeol_str}\t'
                    morph['gr'] = eojeol_str
                else:
                    result_str += f'{row["dp"][e_id - 1][2]}\t'
                    morph['dp'] = row["dp"][e_id - 1][2]
                    if '_' in eojeol_str:
                        result_str += f'{eojeol_str[:eojeol_str.find("_")]}\t'
                        morph['gr'] = eojeol_str[:eojeol_str.find("_")]
                    else:
                        result_str += f'{eojeol_str}\t'
                        morph['gr'] = eojeol_str
                result_str += '\n'
                json_data['data'].append(morph)

            ner_str = ner_str[len(row['pos'][idx][0]):]
            if len(ner_str) == 0 and idx + 1 < len(row['pos']):
                ner_idx += 1
                ner_str = row['ner'][ner_idx][0]
                ner_bio = True
            if row['pos'][idx][0] == ' ' and row['pos'][idx][1] == "SPACE":
                e_id += 1
                eojeol_str = row['dp'][e_id - 1][3]
        return result_str, json_data

DM = DataMaker()

#### Pororo 실행시 에러나는 데이터 필터링

In [ ]:
import pickle
filtered = [] # 성공적 파싱 저장소
error = [] # 에러난거 저장소
# 문장을 하나씩 넣어서 파싱 진행
for _, row in tqdm(crawling_data_df.iterrows(), total=len(crawling_data_df)):
    try:
        filtered.append(DM.parse(row['body']))
    except KeyboardInterrupt:
        break
    except RuntimeError as e:
        error.append((e, row['body']))
        pass
    except ValueError as e:
        error.append((e, row['body']))
        pass
    except IndexError as e:
        error.append((e, row['body']))
        pass
pickle.dump(filtered, open('./output/filtered.pkl', 'wb'))
pickle.dump(error, open('./output/error.pkl', 'wb'))

## 데이터 필터링 및 데이터셋 파일 생성

###  덤프 파일 로드

In [ ]:
import pickle, json
filtered = pickle.load(open('./output/filtered.pkl', 'rb'))
error = pickle.load(open('./output/error.pkl', 'rb'))    
print(len(filtered), filtered[0][0])
print(len(error))

### 사용자 사전에 등록된 개체명만 추출

In [ ]:
import pandas as pd

# 기존의 사용자 사전 데이터 컬럼
# 아티스트(e, k), 배우(e, k), 멤버 -> PER
# 브랜드, 음원플랫폼, 방송플랫폼, 기획사, 팬덤(k,e), 기업 -> ORG
# 한국지명, 외국지명 -> LOC
# 앨범, 곡, 프로그램, 작품 -> AFW
# 수상, 수상2, 행사, 행사2 -> EVT
# 장르, 장르2 -> FLD
# 역할, 기타, 직업 -> 미사용

column_to_ner_tag = {
    '아티스트e': 'PER', '아티스트k': 'PER',
    '배우e': 'PER', '배우k': 'PER', 
    '멤버': 'PER',
    '브랜드': 'ORG', '음원플랫폼': 'ORG', 
    '방송플랫폼': 'ORG', '기획사': 'ORG', 
    '팬덤e': 'ORG', '팬덤k': 'ORG', 
    '기업': 'ORG',
    '한국지명': 'LOC', '외국지명': 'LOC',
    '앨범': 'AFW', '곡': 'AFW', 
    '프로그램': 'AFW', '작품': 'AFW',
    '행사': 'EVT', '행사2': 'EVT',
    '수상': 'EVT', '수상2': 'EVT', 
    '장르': 'FLD', '장르2': 'FLD',
    '기타': 'OTHER', '직업': 'OTHER',
    '역할': 'OTHER'
}
user_dict = {}
user_dict_df = pd.read_csv(filepath_or_buffer='./user-dic/tag_table.tsv', sep='\t', encoding='utf-8')
for col in user_dict_df.columns:
    tag = column_to_ner_tag[col]
    if tag == 'OTHER':
        continue
    for data in user_dict_df[col].dropna():
        user_dict[data.strip()] = tag

### NER 태그 중에서 사용자 사전에 정의된 태그만 필터링

ex) <블랙핑크:ORG>의 앨범 ... -> 블랙핑크 NER 태그가 인식되긴 했지만 PER이 아니므로 일단 필터링

주변 태그들을 확인하고 PER으로 교정할 수 있지만 수량이 많으므로 일단 스킵

In [ ]:
correct_dataset_string = []
correct_json = []

for _, json_data in tqdm(filtered):
    tag_str = ''
    prev_ner = 'O'
    prev_e_id = 0
    eojeol_tag_list = []
    filter_flag = True
    for morph in json_data['data']:
        if tag_str != '' and prev_e_id != morph['e_id']:
            tag_str += ' '
        if prev_e_id != morph['e_id']:
            prev_e_id = morph['e_id']
            eojeol_tag_list.append(morph['ner'])
        if morph['ner'] != 'O':
            if morph['ner'][-1] == 'B':
                tag_str = tag_str.strip()
                if tag_str in user_dict and user_dict[tag_str] != prev_ner:
                    filter_flag = False
                tag_str = morph['morph']
            else:
                tag_str += morph['morph']
        else:
            if tag_str != '':
                tag_str = tag_str.strip()
                if tag_str in user_dict and user_dict[tag_str] != prev_ner:
                    filter_flag = False
            tag_str = morph['morph']
            if tag_str in user_dict and user_dict[tag_str] != prev_ner:
                filter_flag = False
            tag_str = ''
    
    if prev_ner != 'O':
        prev_ner = morph['ner'][:-2]
    if filter_flag:
        correct_dataset_string.append(json_data['RawSentence'] + '\t' +
                                      ' '.join(eojeol_tag_list)
                                      )
        correct_json.append(json_data)
print(len(correct_dataset_string), len(filtered) - len(correct_dataset_string))

In [ ]:
# 필터링된 데이터를 DataFrame으로 변경
ner_tagged_df = pd.DataFrame(
    [x.split('\t') for x in correct_dataset_string],
    columns=['string', 'tag'] 
    )
ner_tagged_df

### Train/Test 데이터 분할

In [ ]:
train_idx = len(ner_tagged_df) // 11
shuffle_ner_tagged_df = ner_tagged_df.sample(frac=1).reset_index(drop=True)
with open('./ner_dataset/test.tsv', 'w', encoding='utf-8') as f:
    for _, row in list(shuffle_ner_tagged_df.iterrows())[:train_idx]:
        f.write(row['string'] + '\t' + row['tag'] + '\n')
with open('./ner_dataset/test.txt', 'w', encoding='utf-8') as f:
    for _, row in list(shuffle_ner_tagged_df.iterrows())[:train_idx]:
        f.write(row['string'] + '\n')
with open('./ner_dataset/train.tsv', 'w', encoding='utf-8') as f:
    for _, row in list(shuffle_ner_tagged_df.iterrows())[train_idx:]:
        f.write(row['string'] + '\t' + row['tag'] + '\n')
print(f'Train Dataset Size: {len(correct_dataset_string) - train_idx}')
print(f'Test Dataset Size : {train_idx}')

# KoBERT-NER 학습

## Train & Eval

Parameters
|Param|Desc|
|:----:|----|
|model_dir|저장된 모델 디렉토리(ner_model)|
|data_dir|데이터셋 디렉토리(ner_dataset)|
|train_batch_size|(Default: 32)|
|eval_batch_size|(Default: 64)|
|num_train_epochs|학습 Epoch(Default: 20)|
|do_train|training 모드|
|do_eval|eval 모드, training 모드와 함께 사용시 1,000 step 마다 Eval|

In [ ]:
!python main.py --data_dir=ner_dataset --model_dir=ner_model --do_train --do_eval --train_batch_size=16 --num_train_epochs=20

## Predict

Parameters
|Param|Desc|
|:----:|----|
|model_dir|저장된 모델 디렉토리(model)|
|input_file|Input 파일명|
|output_file|Output 파일명|
|batch_size|(Default: 32)|

In [ ]:
!python predict.py --model_dir=ner_model --input_file ./dataset/test.txt

## Mecab 설치 및 사용자 사전 적용

In [ ]:
!bash ./user-dic/mecab_install.sh
!bash ./user-dic/apply.sh